<a href="https://colab.research.google.com/github/buthaina279/DS_Capstone_NLP/blob/main/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Libraries

from textblob import TextBlob
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
from google.colab import drive  
#import pycountry
import re
import string

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
# Connect Google Drive to Colab
drive.mount('/content/gdrive')
# Create a variable to store the data path on your drive
path = './gdrive/My Drive/datasets/twitter_analysis'

Mounted at /content/gdrive


In [3]:
saudia_airlines = pd.read_csv("./gdrive/My Drive/datasets/twitter_analysis/saudia_airlines_classified.csv")

In [4]:
saudia_airlines.head(10)

,Unnamed: 0,Unnamed: 0.1,tweet_id,created_at,text,likes,retweeted,location,language,airline,Tweet_nonstop,sentiment_confidence,polarity,sentiment
0,0,14,1335393841810202624,2020-12-06 01:21:16,dear guest there is no flight from india to s...,1,False,Saudi Arabia,en,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.900,0.600,Positive
1,1,23,1335337522738720770,2020-12-05 21:37:29,hello dear guest we hope you check and be inf...,1,False,Saudi Arabia,en,Saudi_airlines,"['', 'hello', 'dear', 'guest', 'hope', 'check'...",0.375,-0.125,Negative
2,2,29,1335334320014315521,2020-12-05 21:24:45,hello dear guest please provide us with the b...,0,False,Saudi Arabia,en,Saudi_airlines,"['', 'hello', 'dear', 'guest', 'please', 'prov...",0.400,0.100,Positive
3,3,38,1335322063851167748,2020-12-05 20:36:03,dear guest there is no flight from india to s...,1,False,Saudi Arabia,en,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.000,0.000,Neutral
4,4,43,1335314759588122626,2020-12-05 20:07:02,hello dear guest you can follow up the flight...,1,False,Saudi Arabia,en,Saudi_airlines,"['', 'hello', 'dear', 'guest', 'follow', 'flig...",0.000,0.000,Neutral
5,5,50,1335306576312356865,2020-12-05 19:34:31,dear guest there is no flight from india to s...,0,False,Saudi Arabia,en,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.000,0.000,Neutral
6,6,51,1335304974679613443,2020-12-05 19:28:09,hello dear valued guest once the confirmation...,1,False,Saudi Arabia,en,Saudi_airlines,"['', 'hello', 'dear', 'valued', 'guest', 'conf...",0.000,0.000,Neutral
7,7,53,1335276146653589507,2020-12-05 17:33:36,dear guest there is no flight from india to s...,0,False,Saudi Arabia,en,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.000,0.000,Neutral
8,8,55,1335275872295804928,2020-12-05 17:32:30,dear guest there is no flight from india to s...,0,False,Saudi Arabia,en,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.000,0.000,Neutral
9,9,57,1335266364739694593,2020-12-05 16:54:44,hello dear guest imran please provide us with...,0,False,Saudi Arabia,en,Saudi_airlines,"['', 'hello', 'dear', 'guest', 'imran', 'pleas...",0.400,0.100,Positive


#preproccesing

Dealing with dates and time

#Extracting Features from Cleaned Tweets

https://www.analyticsvidhya.com/blog/2018/07/hands-on-sentiment-analysis-dataset-python/

#Bag-of-Words Features

Bag-of-Words is a method to represent text into numerical features


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(saudia_airlines['Tweet_nonstop'])

In [ ]:
bow

#TF-IDF Features

This is another method which is based on the frequency method but it is different to the bag-of-words approach in the sense that it takes into account, not just the occurrence of a word in a single document (or tweet) but in the entire corpus.

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(saudia_airlines['Tweet_nonstop'])

#Model calibration

https://towardsdatascience.com/classifier-calibration-7d0be1e05452

In [11]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

X = saudia_airlines.iloc[:,:-1].values #all predictor columns without the target
y = saudia_airlines.iloc[:,-1].values #last column (target)

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y, random_state=123)

Next, we instantiate two classifiers to compare; a simple logistic regression model and an implementation of Support Vector Machines provided by scikit-learn.

In [12]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


lgr = LogisticRegression(C=1, solver='lbfgs')
svc = SVC(max_iter=10000, probability=True)

Finally, we fit the classifiers to our training data and compute our predictions on the test data set. Specifically for the SVM, to get the probabilities for the positive class we need to know how the decision function separates the test samples, and normalize the results to be between `0` and `1`.

In [13]:
probs_lgr = lgr.fit(X_train, y_train).predict_proba(X_test)[:,1]
preds_svc = svc.fit(X_train, y_train).predict(X_test)

probs_svc = svc.decision_function(X_test)
probs_svc = (probs_svc - probs_svc.min()) / (probs_svc.max() - probs_svc.min())

ValueError: ignored

Let us now plot the Kernel Density Estimation for the two classifiers.

In [ ]:
plt.figure(figsize=(10,6))
sns.kdeplot(probs_lgr, label='Logistic regression')
sns.kdeplot(preds_svc, label='SVM')
plt.title("Probability Density Plot for 2 Classifiers")
plt.show()

let us check the AUC-ROC curve for the two binary classifiers, but this time using the probabilities we calculated for the SVM.

In [ ]:
from sklearn import metrics


plt.figure(figsize=(8,6))
plt.plot([0, 1], [0, 1],'r--')

pred = probs_lgr
label = y_test
fpr, tpr, thresh = metrics.roc_curve(label, pred)
auc = metrics.roc_auc_score(label, pred)
plt.plot(fpr, tpr, label=f'Logistic regression, auc = {str(round(auc,3))}')

pred = probs_svc
fpr, tpr, thresh = metrics.roc_curve(label, pred)
auc = metrics.roc_auc_score(label, pred)
plt.plot(fpr, tpr, label=f'SVC, auc = {str(round(auc,3))}')

plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.title("AUC-ROC for two models")
plt.legend()
plt.show()

To plot the calibration curve of each classifier we define a utility function like the one below.


In [ ]:
from sklearn.calibration import calibration_curve


def plot_calibration_curve(name, fig_index, probs):
    """Plot calibration curve for est w/o and with calibration. """

    fig = plt.figure(fig_index, figsize=(10, 10))
    ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    ax2 = plt.subplot2grid((3, 1), (2, 0))
    
    ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
    
    frac_of_pos, mean_pred_value = calibration_curve(y_test, probs, n_bins=10)

    ax1.plot(mean_pred_value, frac_of_pos, "s-", label=f'{name}')
    ax1.set_ylabel("Fraction of positives")
    ax1.set_ylim([-0.05, 1.05])
    ax1.legend(loc="lower right")
    ax1.set_title(f'Calibration plot ({name})')
    
    ax2.hist(probs, range=(0, 1), bins=10, label=name, histtype="step", lw=2)
    ax2.set_xlabel("Mean predicted value")
    ax2.set_ylabel("Count")

In [ ]:
# plot calibration curve for logistic regression
plot_calibration_curve("Logistic regression", 1, probs_lgr)

In [ ]:
# plot calibration curve for the SVM
plot_calibration_curve("SVM", 1, probs_svc)

#Calibrating the model

The two most popular methods of calibrating a machine learning model are the isotonic and Platt's method.

In [ ]:
from sklearn.calibration import CalibratedClassifierCV


lgr = LogisticRegression(C=1, solver='lbfgs')
svc = SVC(max_iter=10000, probability=True)

platts_scaling = CalibratedClassifierCV(svc, cv=2, method='sigmoid')
platts_scaling.fit(X_train, y_train)
calibrated_probs = platts_scaling.predict_proba(X_test)[:,1]

plot_calibration_curve("SVM", 3, calibrated_probs)